In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
import plotly.express as px

In [ ]:
### DATA IMPORT ###
Data = pd.read_csv('First Project/stock_market_data.csv')
AMZN = Data[:3119]
FB = Data[3119:6238]
TSLA = Data[3119:9357]
GOOGL = Data[9357:12476]
AAPL = Data[12476:]
tlist = [AMZN, FB, TSLA, GOOGL, AAPL]
tickers = ['AMZN', 'FB', 'TSLA', 'GOOGL', 'AAPL']

In [ ]:
### USER DATABASE ###
user_shares = {}

Welcome to NYSE Stock market, please sign up to continue:

please enter your first name and last name:

In [ ]:
name = input()

please enter your ID number:

In [ ]:
ID = input()

please enter you birth date in this format(dd/mm/yyyy):

In [ ]:
BD = input()

please enter your balance in US$:

In [ ]:
Balance = int(input())

Thanks for your registration, now you have access to trade in Stock market

to buy shares:

In [ ]:
print(f'Your Balance is : {Balance}')
pick = int(input('''to buy shares, please enter the ticker's number:
1- AMZN
2- FB
3- TSLA
4- GOOGL
5- AAPL
'''))
curr_price = list(tlist[pick-1]['Open'])[-1]
to_buy = float(input(f'{tickers[pick-1]} price is {curr_price} per share, how many shares do you want to buy?'))
if(to_buy * curr_price > Balance):
    print("Insufficient Balance")
else:
    Balance -= to_buy * curr_price
    if tickers[pick-1] in user_shares.keys():
        user_shares[tickers[pick-1]] += to_buy
    else:
        user_shares[tickers[pick-1]] = to_buy
    print(f'You have successfully buy {to_buy} shares of {tickers[pick-1]}, your new Balance is : {Balance}')
        

to sell shares:

In [ ]:
d = {'Shares': list(user_shares.keys()), 'Quantities': list(user_shares.values())}
df2 = pd.DataFrame(data=d)
print(df2)
print(f'Your Shares are listed Below :')
pick = int(input('''to sell shares, please enter the ticker's number:
1- AMZN
2- FB
3- TSLA
4- GOOGL
5- AAPL
'''))
if tickers[pick-1] in user_shares.keys():   
    curr_price = list(tlist[pick-1]['Open'])[-1]
    to_sell = float(input(f'{tickers[pick-1]} price is {curr_price} per share and you have {user_shares[tickers[pick-1]]} shares, how many shares of yours do you want to sell?'))
    if(to_sell > float(user_shares[tickers[pick-1]])):
        print("Insufficient Shares!")
    else:
        user_shares[tickers[pick-1]] -= to_sell
        Balance += to_sell * curr_price
        print(f'You have successfully sell {to_sell} shares of {tickers[pick-1]}, your new Balance is : {Balance}')
else:
    print(f'You do not have any {tickers[pick-1]} shares in your portfolio!')
        

to check charts:

In [ ]:
num = int(input('''to check the charts, please enter the ticker's number:
1- AMZN
2- FB
3- TSLA
4- GOOGL
5- AAPL
'''))
if num in range(1,6):
    fig = px.line(tlist[num-1], x = 'Date', y = 'Open', title=f'{tickers[num-1]} Share Prices')
    fig.show()
    xax = list(tlist[num-1]['Date'])
    yax = list(tlist[num-1]['Open'])
    xs = [datetime.strptime(x, "%Y-%m-%d") for x in xax]
    plt.plot(xs, yax)
    plt.show()
else:
    print("Wrong ticker number!")